db에서 불러오기

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql
# from sqlalchemy import create_engine

In [8]:
# 1. 데이터베이스 연결 -> 연결 객체 반환
conn = pymysql.connect(host="localhost", port=3306, db="mp",
                       user="root", passwd="1234")

# 2. 명령 도구 만들기
cursor = conn.cursor()

In [9]:
# 3. SQL 작성 + 명령 실행 ( 조회 명령 )
sql = """select * from fine_dust"""
cursor.execute(sql)

# 4. 조회 결과 읽기
fd_df = cursor.fetchall()

# 5. 데이터 처리 (표시)
for fd in fd_df[:10]:
    print(fd)

(108, '서울', datetime.datetime(2021, 1, 1, 0, 0), 36)
(108, '서울', datetime.datetime(2021, 1, 2, 0, 0), 43)
(108, '서울', datetime.datetime(2021, 1, 3, 0, 0), 47)
(108, '서울', datetime.datetime(2021, 1, 4, 0, 0), 50)
(108, '서울', datetime.datetime(2021, 1, 5, 0, 0), 37)
(108, '서울', datetime.datetime(2021, 1, 6, 0, 0), 30)
(108, '서울', datetime.datetime(2021, 1, 7, 0, 0), 56)
(108, '서울', datetime.datetime(2021, 1, 8, 0, 0), 27)
(108, '서울', datetime.datetime(2021, 1, 9, 0, 0), 33)
(108, '서울', datetime.datetime(2021, 1, 10, 0, 0), 48)
